In [4]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from matplotlib.colors import LinearSegmentedColormap
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Activation, Add, Dense, Conv2D, GlobalAveragePooling2D, MaxPooling2D
from keras.layers import BatchNormalization, Dropout
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.models import Model
from keras.utils import plot_model
from sklearn.metrics import classification_report, confusion_matrix

2024-03-31 15:49:39.633926: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-31 15:49:39.634051: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-31 15:49:39.762281: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
dataset_path='/kaggle/input/dataset/Garbage classification'
garbage_types = os.listdir(dataset_path)


In [7]:
# Here I'm going to create the dataframe for thr image ---> Basically, a image pipeline
data = []
for garbage_type in garbage_types:
    for file in os.listdir(os.path.join(dataset_path, garbage_type)):
        data.append((os.path.join(dataset_path, garbage_type, file), garbage_type))
df = pd.DataFrame(data, columns=['filepath', 'label'])
df.head()


,filepath,label
0,/kaggle/input/dataset/Garbage classification/m...,metal
1,/kaggle/input/dataset/Garbage classification/m...,metal
2,/kaggle/input/dataset/Garbage classification/m...,metal
3,/kaggle/input/dataset/Garbage classification/m...,metal
4,/kaggle/input/dataset/Garbage classification/m...,metal


In [8]:
# SPlit into different train and test dataset
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# Print the number of images in each set
print(f"Number of images in the training set: {len(train_df)}")
print(f"Number of images in the validation set: {len(val_df)}")

Number of images in the training set: 2021
Number of images in the validation set: 506


Here I will Perfome the Data Augumentation to increase the size of the daata

In [9]:
train_datagen = ImageDataGenerator(
    rescale=1./255,                     
    rotation_range=45,                 
    width_shift_range=0.15,             
    height_shift_range=0.15,            
    zoom_range=0.15,                   
    horizontal_flip=True,             
    vertical_flip=True,                 
    shear_range=0.05,                   
    brightness_range=[0.9, 1.1],       
    channel_shift_range=10,             
    fill_mode='nearest'   
)    
                   
val_datagen =ImageDataGenerator(rescale=1./255)   

Here I will create the Batches of the Training Dataset and Testing Dataset because the Image Pipeline is big and Batch Processing will help

In [10]:
# For Training Data
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,                  
    x_col="filepath",                    
    y_col="label",                        
    target_size=(384, 384),              
    batch_size=32,                       
    class_mode='categorical',            
    seed=42,                             
    shuffle=False                       
)

# For Testing Data
val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,                    
    x_col="filepath",                    
    y_col="label",                       # Column with image labels
    target_size=(384, 384),              # Resize all images to size of 384x384
    batch_size=32,                       # Number of images per batch
    class_mode='categorical',            # One-hot encode labels
    seed=42,                             # Seed for random number generator to ensure reproducibility
    shuffle=False                        # Data is not shuffled; order retained from DataFrame
)

len(train_generator)
len(val_generator)

Found 2021 validated image filenames belonging to 6 classes.
Found 506 validated image filenames belonging to 6 classes.


16

In [11]:
# CBAM Definition
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, Reshape, Dense, Multiply, Conv2D, Add, Activation, BatchNormalization

class ChannelAttention(tf.keras.layers.Layer):
    def __init__(self, ratio=8):
        super(ChannelAttention, self).__init__()
        self.avg_pool = GlobalAveragePooling2D()
        self.max_pool = GlobalMaxPooling2D()
        self.shared_mlp = Dense(units=1, activation='relu', use_bias=True, kernel_initializer='he_normal')
        self.ratio = ratio

    def call(self, x):
        avg_pool = self.avg_pool(x)
        max_pool = self.max_pool(x)
        channel_attentions = self.shared_mlp(tf.concat([avg_pool, max_pool], axis=1))
        return Multiply()([x, Activation('sigmoid')(channel_attentions)])

class SpatialAttention(tf.keras.layers.Layer):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv = Conv2D(filters=1, kernel_size=kernel_size, padding='same', activation='sigmoid', kernel_initializer='he_normal')

    def call(self, x):
        avg_pool = tf.reduce_mean(x, axis=-1, keepdims=True)
        max_pool = tf.reduce_max(x, axis=-1, keepdims=True)
        combined = tf.concat([avg_pool, max_pool], axis=-1)
        return Multiply()([x, self.conv(combined)])

class CBAM(tf.keras.layers.Layer):
    def __init__(self, ratio=8, kernel_size=7):
        super(CBAM, self).__init__()
        self.channel_attention = ChannelAttention(ratio)
        self.spatial_attention = SpatialAttention(kernel_size)

    def call(self, x):
        x_out = self.channel_attention(x)
        return self.spatial_attention(x_out)

Now I will create the model the plain model of ResNet-50 and will further modify it according to the requirement

In [33]:
def residual_block(X, kernel_size, filters, reduce=False, stride=2):
    """
    Implement a residual block for ResNet architectures.
    
    Arguments:
    X           -- input tensor of shape (m, height, width, channels)
    kernel_size -- integer, kernel size of the middle convolutional layer in the main path
    filters     -- python list of integers, defining the number of filters in the CONV layers of the main path
    reduce      -- boolean, whether to reduce the spatial dimensions and increase depth; 
                    if True, applies 1x1 CONV layer to the shortcut path.
    stride      -- integer, strides for the convolutional layer
    
    Returns:
    X           -- output of the identity block, tensor of shape (height, width, channels)
    """
     # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. We will need this later to add back to the main path. 
    X_shortcut = X
    
    if reduce:
        # if we are to reduce the spatial size, apply a 1x1 CONV layer to the shortcut path
        #First Component of main path
        X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (stride,stride), padding = 'valid', kernel_initializer='he_normal')(X)
        X = BatchNormalization(axis = 3)(X)
        X = Activation('relu')(X)
         
        X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (stride,stride), padding = 'valid', kernel_initializer='he_normal')(X_shortcut)
        X_shortcut = BatchNormalization(axis = 3)(X_shortcut)
    else: 
        # First component of main path
        X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', kernel_initializer='he_normal')(X)
        X = BatchNormalization(axis = 3)(X)
        X = Activation('relu')(X)
        
        
        # Second component of main path
    X = Conv2D(filters = F2, kernel_size = (kernel_size, kernel_size), strides = (1,1), padding = 'same', kernel_initializer='he_normal')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
     # Third component of main path
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', kernel_initializer='he_normal')(X)
    X = BatchNormalization(axis = 3)(X)
    X = CBAM()(X)

    # Final step: Add shortcut value to main path, and pass it through a ReLU activation 
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    
    return X



In [9]:
# Original ResNet-50 Definition
# def ResNet_50(input_shape, classes):
    
    

#     # Define the input as a tensor with shape input_shape
#     X_input = Input(input_shape)
#      # Block 1
#     X = Conv2D(64, (7, 7), strides=(2, 2), kernel_initializer='he_normal')(X_input)
#     X = BatchNormalization(axis=3)(X)
#     X = Activation('relu')(X)
#     X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    

    # Block 2
#     X = residual_block(X, 3, [64, 64, 256], reduce=True, stride=1)
#     X = residual_block(X, 3, [64, 64, 256])
#     X = residual_block(X, 3, [64, 64, 256])
    # Block 3 
#     X = residual_block(X, 3, [128, 128, 512], reduce=True, stride=2)
#     X = residual_block(X, 3, [128, 128, 512])
#     X = residual_block(X, 3, [128, 128, 512])
#     X = residual_block(X, 3, [128, 128, 512])

    # Block 4 
#     X = residual_block(X, 3, [256, 256, 1024], reduce=True, stride=2)
#     X = residual_block(X, 3, [256, 256, 1024])
#     X = residual_block(X, 3, [256, 256, 1024])
#     X = residual_block(X, 3, [256, 256, 1024])
#     X = residual_block(X, 3, [256, 256, 1024])
#     X = residual_block(X, 3, [256, 256, 1024])
     # Block 5 
#     X = residual_block(X, 3, [512, 512, 2048], reduce=True, stride=2)
#     X = residual_block(X, 3, [512, 512, 2048])
#     X = residual_block(X, 3, [512, 512, 2048])

    # Global Average Pooling to reduce spatial dimensions
#     X = GlobalAveragePooling2D()(X)
    
    # Fully Connected Layer for classification
#     X = Dense(classes, activation='softmax')(X)
        
    # Create the model
#     model = Model(inputs = X_input, outputs = X, name='ResNet50')

#     return model

In [34]:
def Modified_ResNet50(input_shape, classes):
    """
    Arguments:
    input_shape -- tuple shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)
      # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), kernel_initializer='he_normal')(X_input)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = residual_block(X, 3, [64, 64, 256], reduce=True, stride=1)
    X = residual_block(X, 3, [64, 64, 256])
    X = residual_block(X, 3, [64, 64, 256])

    # Stage 3 
    X = residual_block(X, 3, [128, 128, 512], reduce=True, stride=2)
    X = residual_block(X, 3, [128, 128, 512])
    X = residual_block(X, 3, [128, 128, 512])
    X = residual_block(X, 3, [128, 128, 512])
      # Stage 4 
    X = residual_block(X, 3, [256, 256, 1024], reduce=True, stride=2)
    X = residual_block(X, 3, [256, 256, 1024])
    X = residual_block(X, 3, [256, 256, 1024])
    X = residual_block(X, 3, [256, 256, 1024])
    X = residual_block(X, 3, [256, 256, 1024])
    X = residual_block(X, 3, [256, 256, 1024])

    # Stage 5 
    X = residual_block(X, 3, [512, 512, 2048], reduce=True, stride=2)
    X = residual_block(X, 3, [512, 512, 2048])
    X = residual_block(X, 3, [512, 512, 2048])
    
      # Global Average Pooling to reduce spatial dimensions
    X = GlobalAveragePooling2D()(X)
    
    # Add Dropout to prevent overfitting
    X = Dropout(0.5)(X)
    
    # Fully Connected Layer for classification
    X = Dense(classes, activation='softmax')(X)
        
    # Create the model
    model = Model(inputs = X_input, outputs = X, name='Modified_ResNet50')

    return model



In [11]:

# CBAM Module Definition
# import tensorflow as tf

# class ChannelAttention(tf.keras.layers.Layer):
#     def __init__(self, channel_in, reduction_ratio=16):
#         super(ChannelAttention, self).__init__()
#         self.shared_mlp = tf.keras.Sequential([
#             tf.keras.layers.GlobalAveragePooling2D(),
#             tf.keras.layers.Dense(units=channel_in // reduction_ratio, activation='relu'),
#             tf.keras.layers.Dense(units=channel_in, activation='sigmoid')
#         ])

#     def call(self, x):
#         avg_pool = self.avg_pool(x)
#         max_pool = self.max_pool(x)
#         channel_attentions = self.shared_mlp(tf.concat([avg_pool, max_pool], axis=1))
#         channel_attentions = tf.expand_dims(tf.expand_dims(channel_attentions, axis=1), axis=1)  # Add dimensions for broadcasting
#         return x * channel_attentions

# class SpatialAttention(tf.keras.layers.Layer):
#     def __init__(self, kernel_size=7):
#         super(SpatialAttention, self).__init__()
#         self.compress = tf.keras.layers.Lambda(lambda x: tf.reduce_max(x, axis=3, keepdims=True) + tf.reduce_mean(x, axis=3, keepdims=True))
#         self.spatial_attention = tf.keras.Sequential([
#             tf.keras.layers.Conv2D(filters=1, kernel_size=kernel_size, padding='same', use_bias=False),
#             tf.keras.layers.BatchNormalization(),
#             tf.keras.layers.Activation('sigmoid')
#         ])

#     def call(self, x):
#         x_compress = self.compress(x)
#         x_output = self.spatial_attention(x_compress)
#         return x * x_output

# class CBAM(tf.keras.Model):
#     def __init__(self, channel_in, reduction_ratio=16, spatial=True):
#         super(CBAM, self).__init__()
#         self.spatial = spatial
#         self.channel_attention = ChannelAttention(channel_in, reduction_ratio=reduction_ratio)
#         if self.spatial:
#             self.spatial_attention = SpatialAttention()

#     def call(self, x):
#         x_out = self.channel_attention(x)
#         if self.spatial:
#             x_out = self.spatial_attention(x_out)
#         return x_out


In [12]:
# import tensorflow as tf

# class BottleNeck(tf.keras.layers.Layer):
#     def __init__(self, in_channels, out_channels, expansion=4, stride=1, use_cbam=True):
#         super(BottleNeck, self).__init__()

#         self.use_cbam = use_cbam
#         self.conv1 = tf.keras.layers.Conv2D(filters=out_channels, kernel_size=1, strides=stride, padding='same', use_bias=False)
#         self.bn1 = tf.keras.layers.BatchNormalization()
#         self.conv2 = tf.keras.layers.Conv2D(filters=out_channels, kernel_size=3, strides=1, padding='same', use_bias=False)
#         self.bn2 = tf.keras.layers.BatchNormalization()
#         self.conv3 = tf.keras.layers.Conv2D(filters=out_channels*expansion, kernel_size=1, strides=1, padding='same', use_bias=False)
#         self.bn3 = tf.keras.layers.BatchNormalization()
#         self.relu = tf.keras.layers.ReLU()

#         self.identity_connection = tf.keras.Sequential()
#         if stride != 1 or in_channels != expansion*out_channels:
#             self.identity_connection = tf.keras.Sequential([
#                 tf.keras.layers.Conv2D(filters=out_channels*expansion, kernel_size=1, strides=stride, padding='same', use_bias=False),
#                 tf.keras.layers.BatchNormalization()
#             ])

#         if self.use_cbam:
#             self.cbam = CBAM(out_channels*expansion)

#     def call(self, inputs):
#         x = self.relu(self.bn1(self.conv1(inputs)))
#         x = self.relu(self.bn2(self.conv2(x)))
#         x = self.bn3(self.conv3(x))

#         if self.use_cbam:
#             x = self.cbam(x)

#         x += self.identity_connection(inputs)
#         x = self.relu(x)

#         return x

# class ResNet50(tf.keras.Model):
#     def __init__(self, use_cbam=True, image_depth=3, num_classes=6):
#         super(ResNet50, self).__init__()

#         self.in_channels = 64
#         self.expansion = 4
#         self.num_blocks = [3, 3, 3, 2]

#         self.conv_block1 = tf.keras.Sequential([
#             tf.keras.layers.Conv2D(filters=self.in_channels, kernel_size=7, strides=2, padding='same', use_bias=False),
#             tf.keras.layers.BatchNormalization(),
#             tf.keras.layers.ReLU(),
#             tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')
#         ])

#         self.layer1 = self.make_layer(out_channels=64, num_blocks=self.num_blocks[0], stride=1, use_cbam=use_cbam)
#         self.layer2 = self.make_layer(out_channels=128, num_blocks=self.num_blocks[1], stride=2, use_cbam=use_cbam)
#         self.layer3 = self.make_layer(out_channels=256, num_blocks=self.num_blocks[2], stride=2, use_cbam=use_cbam)
#         self.layer4 = self.make_layer(out_channels=512, num_blocks=self.num_blocks[3], stride=2, use_cbam=use_cbam)
#         self.avgpool = tf.keras.layers.GlobalAveragePooling2D()
#         self.linear = tf.keras.layers.Dense(units=num_classes)

#     def make_layer(self, out_channels, num_blocks, stride, use_cbam):
#         strides = [stride] + [1]*(num_blocks-1)
#         layers = []
#         for stride in strides:
#             layers.append(BottleNeck(in_channels=self.in_channels, out_channels=out_channels, stride=stride, expansion=self.expansion, use_cbam=use_cbam))
#             self.in_channels = out_channels * self.expansion
#         return tf.keras.Sequential(layers)

#     def call(self, inputs):
#         x = self.conv_block1(inputs)
#         x = self.layer1(x)
#         x = self.layer2(x)
#         x = self.layer3(x)
#         x_conv = self.layer4(x)
#         x = self.avgpool(x_conv)
#         x = tf.keras.layers.Flatten()(x)
#         x = self.linear(x)
#         return x_conv, x


In [13]:
# # Define the shape of the input images and number of classes
# input_shape = (384, 384, 3)
# num_classes = 6

# # Initialize the modified ResNet50 model with the specified parameters
# modified_resnet50_model = Modified_ResNet50(input_shape=input_shape, classes=num_classes)

In [14]:
# modified_resnet50_model.summary()

In [36]:
# New Method
# import tensorflow as tf
# from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, Reshape, Dense, Multiply, Conv2D, Add, Activation, BatchNormalization

# class ChannelAttention(tf.keras.layers.Layer):
#     def __init__(self, ratio=8):
#         super(ChannelAttention, self).__init__()
#         self.avg_pool = GlobalAveragePooling2D()
#         self.max_pool = GlobalMaxPooling2D()
#         self.shared_mlp = Dense(units=1, activation='relu', use_bias=True, kernel_initializer='he_normal')
#         self.ratio = ratio

#     def call(self, x):
#         avg_pool = self.avg_pool(x)
#         max_pool = self.max_pool(x)
#         channel_attentions = self.shared_mlp(tf.concat([avg_pool, max_pool], axis=1))
#         return Multiply()([x, Activation('sigmoid')(channel_attentions)])

# class SpatialAttention(tf.keras.layers.Layer):
#     def __init__(self, kernel_size=7):
#         super(SpatialAttention, self).__init__()
#         self.conv = Conv2D(filters=1, kernel_size=kernel_size, padding='same', activation='sigmoid', kernel_initializer='he_normal')

#     def call(self, x):
#         avg_pool = tf.reduce_mean(x, axis=-1, keepdims=True)
#         max_pool = tf.reduce_max(x, axis=-1, keepdims=True)
#         combined = tf.concat([avg_pool, max_pool], axis=-1)
#         return Multiply()([x, self.conv(combined)])

# class CBAM(tf.keras.layers.Layer):
#     def __init__(self, ratio=8, kernel_size=7):
#         super(CBAM, self).__init__()
#         self.channel_attention = ChannelAttention(ratio)
#         self.spatial_attention = SpatialAttention(kernel_size)

#     def call(self, x):
#         x_out = self.channel_attention(x)
#         return self.spatial_attention(x_out)

# def residual_block(input_tensor, filters, kernel_size=3, stride=1, use_cbam=True):
#     x = Conv2D(filters=filters, kernel_size=kernel_size, strides=stride, padding='same', kernel_initializer='he_normal')(input_tensor)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)

#     x = Conv2D(filters=filters, kernel_size=kernel_size, strides=1, padding='same', kernel_initializer='he_normal')(x)
#     x = BatchNormalization()(x)

#     if use_cbam:
#         x = CBAM()(x)

#     shortcut = input_tensor
#     if stride != 1 or input_tensor.shape[-1] != filters:
#         shortcut = Conv2D(filters=filters, kernel_size=1, strides=stride, padding='same', kernel_initializer='he_normal')(input_tensor)
#         shortcut = BatchNormalization()(shortcut)

#     x = Add()([x, shortcut])
#     x = Activation('relu')(x)
#     return x

# def Modified_ResNet50(input_shape=(224, 224, 3), classes=1000):
#     input_data = tf.keras.Input(shape=input_shape)
#     X = Conv2D(64, (7, 7), strides=(2, 2), padding='same')(input_data)
#     X = BatchNormalization()(X)
#     X = Activation('relu')(X)
#     X = MaxPooling2D((3, 3), strides=(2, 2))(X)
#     X = CBAM()(X)
#     X = residual_block(X, 64, use_cbam=True)
#     X = residual_block(X, 64, use_cbam=True)
#     X = residual_block(X, 64, use_cbam=True)

#     X = residual_block(X, 128, stride=2, use_cbam=True)
#     X = residual_block(X, 128, use_cbam=True)
#     X = residual_block(X, 128, use_cbam=True)
#     X = residual_block(X, 128, use_cbam=True)

#     X = residual_block(X, 256, stride=2, use_cbam=True)
#     X = residual_block(X, 256, use_cbam=True)
#     X = residual_block(X, 256, use_cbam=True)
#     X = residual_block(X, 256, use_cbam=True)
#     X = residual_block(X, 256, use_cbam=True)
#     X = residual_block(X, 256, use_cbam=True)

#     X = residual_block(X, 512, stride=2, use_cbam=True)
#     X = residual_block(X, 512, use_cbam=True)
#     X = residual_block(X, 512, use_cbam=True)

#     X = GlobalAveragePooling2D()(X)
#     output = Dense(classes, activation='softmax')(X)
#     model = tf.keras.Model(inputs=input_data, outputs=output)
#     return model

# # Create the model
# input_shape = (384,384, 3)
# num_classes = 6
# modified_resnet50_model = Modified_ResNet50(input_shape=input_shape, classes=num_classes)

# Print model summary
# modified_resnet50_model.summary()


In [14]:
#  ReduceLROnPlateau 
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=15, min_lr=0.00001)

#  EarlyStopping 
early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=50, restore_best_weights=True, verbose=1)

In [15]:
class_labels = train_df['label'].unique()
class_labels

array(['trash', 'glass', 'paper', 'metal', 'plastic', 'cardboard'],
      dtype=object)

In [16]:
train_generator.class_indices

{'cardboard': 0, 'glass': 1, 'metal': 2, 'paper': 3, 'plastic': 4, 'trash': 5}

In [17]:
weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=train_df['label'])
weights

array([3.06212121, 0.83998337, 0.70912281, 1.02693089, 0.87489177,
       1.04606625])

In [18]:
class_weights = dict(zip(train_generator.class_indices.values(),weights))
class_weights

{0: 3.062121212121212,
 1: 0.8399833748960931,
 2: 0.7091228070175438,
 3: 1.026930894308943,
 4: 0.8748917748917749,
 5: 1.0460662525879918}

In [15]:
# input_shape = (384, 384, 3)
# num_classes = 6

# # Initialize the modified ResNet50 model with the specified parameters
# resnet50_model = ResNet50(input_shape,num_classes)

In [22]:
# resnet50_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
# num_epochs = 10

# # Train the model
# history = resnet50_model.fit(train_generator, 
#                                       steps_per_epoch=len(train_generator), 
#                                       epochs=num_epochs, 
#                                       validation_data=val_generator, 
#                                       validation_steps=len(val_generator),
#                                       class_weight=class_weights,
#                                       callbacks=[reduce_lr, early_stopping])

In [35]:
# Define the shape of the input images and number of classes
input_shape = (384, 384, 3)
num_classes = 6

# Initialize the modified ResNet50 model with the specified parameters
modified_resnet50_model = Modified_ResNet50(input_shape=input_shape, classes=num_classes)

In [27]:
modified_resnet50_model.summary()

NameError: name 'modified_resnet50_model' is not defined

In [29]:
modified_resnet50_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [54]:

num_epochs = 100

# Train the model
history = modified_resnet50_model.fit(train_generator, 
                                      steps_per_epoch=len(train_generator), 
                                      epochs=num_epochs, 
                                      validation_data=val_generator, 
                                      validation_steps=len(val_generator),
                                      class_weight=class_weights,
                                      callbacks=[reduce_lr, early_stopping])

Epoch 1/100


I0000 00:00:1711892358.783512     165 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


64/64 ━━━━━━━━━━━━━━━━━━━━ 188s 2s/step - accuracy: 0.3712 - loss: 1.7516 - val_accuracy: 0.1917 - val_loss: 2.2906 - learning_rate: 0.0010
Epoch 2/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - learning_rate: 0.0010
Epoch 3/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 73s 1s/step - accuracy: 0.4584 - loss: 1.4040 - val_accuracy: 0.1917 - val_loss: 1.8676 - learning_rate: 0.0010
Epoch 4/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 325us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - learning_rate: 0.0010
Epoch 5/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 73s 1s/step - accuracy: 0.5119 - loss: 1.2612 - val_accuracy: 0.2095 - val_loss: 2.1114 - learning_rate: 0.0010
Epoch 6/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 332us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - learning_rate: 0.0010
Epoch 7/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 73s 1s/step - 